In [1]:
import pandas as pd
import os
import sys
import logging
import pandas as pd
from google.cloud import bigquery
from hashlib import md5
from typing import List
import shortuuid
import numpy

In [ ]:
# df = pd.read_csv("./spreadspoke_scores.csv")

# df = df[df["schedule_season"] > 1978]

# df = df.drop(columns="weather_detail")

# display(df[df["schedule_playoff"] == True])

In [7]:
DATA_DIR = "./spreadspoke_scores.csv"

PROJECT_NAME = "team-week-10"
DATASET_NAME = "nfl_sports_betting"

TABLE_METADATA = {
    "point_spread": {
        "table_name": "fct_points_spread",
        "schema": [
            bigquery.SchemaField("uuid", "string", mode="REQUIRED"),
            bigquery.SchemaField("team_fav_id", "string", mode="REQUIRED"),
            bigquery.SchemaField("home_team", "string", mode="REQUIRED"),
            bigquery.SchemaField("away_team", "string", mode="Required"),
            bigquery.SchemaField("spread_favorite", "float69", mode="Required"),
            bigquery.SchemaField("home_away_diff", "int", mode="Required"),
        ]
    },
    "over_under": {
        "table_name": "fct_over_under",
        "schema": [
            bigquery.SchemaField("uuid", "string", mode="REQUIRED"),
            bigquery.SchemaField("over_under_line", "string", mode="REQUIRED"),
            bigquery.SchemaField("total_score", "int", mode="REQUIRED"),
            bigquery.SchemaField("team_fav_id", "string", mode="REQUIRED"),
            bigquery.SchemaField("home_team", "string", mode="REQUIRED"),
            bigquery.SchemaField("away_team", "string", mode="REQUIRED"),
            bigquery.SchemaField("over_or_under", "string", mode="REQUIRED"),
        ]
    },
    "games": {
        "table_name": "dim_games",
        "schema": [
            bigquery.SchemaField("date", "date", mode="REQUIRED"),
            bigquery.SchemaField("season", "int64", mode="REQUIRED"),
            bigquery.SchemaField("week", "string", mode="REQUIRED"),
            bigquery.SchemaField("playoff", "boolean", mode="REQUIRED"),
            bigquery.SchemaField("team_home", "string", mode="REQUIRED"),
            bigquery.SchemaField("score_home", "int64", mode="REQUIRED"),
            bigquery.SchemaField("score_away", "int64", mode="REQUIRED"),
            bigquery.SchemaField("team_away", "string", mode="REQUIRED"),
            bigquery.SchemaField("stadium", "string", mode="REQUIRED"),
            bigquery.SchemaField("temperature_F", "float64", mode="NULLABLE"),
            bigquery.SchemaField("wind_mph", "float64", mode="NULLABLE"),
            bigquery.SchemaField("humidity_%", "float64", mode="NULLABLE"),
            bigquery.SchemaField("uuid", "string", mode="REQUIRED")
        ]
    },
    "lookup_teams": {
        "table_name": "lookup_teams",
        "schema": [
            bigquery.SchemaField("team_id", "string", mode="REQUIRED"),
            bigquery.SchemaField("team_name", "string", mode="REQUIRED"),
            bigquery.SchemaField("conference_string", "string", mode="REQUIRED"),
            bigquery.SchemaField("division", "string", mode="REQUIRED"),
        ]
    }
}

logging.basicConfig(
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)

In [3]:
dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
client = bigquery.Client()

def create_dataset(client: bigquery.Client, dataset_id: str, location: str = "US"):
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = location
    dataset = client.create_dataset(dataset, exists_ok=True)
    logger.info(f"Created dataset: {dataset.full_dataset_id}")

create_dataset(client, dataset_id)

[DEBUG][2024-01-10 13:11:15,425][_default:0255] : Checking /home/jarvismwynn/.creds/jarvis.json for explicit credentials as part of auth process...
[DEBUG][2024-01-10 13:11:15,428][_default:0255] : Checking /home/jarvismwynn/.creds/jarvis.json for explicit credentials as part of auth process...
[DEBUG][2024-01-10 13:11:15,430][opentelemetry_tracing:0066] : This service is instrumented using OpenTelemetry. OpenTelemetry or one of its components could not be imported; please add compatible versions of opentelemetry-api and opentelemetry-instrumentation packages in order to get BigQuery Tracing data.
[DEBUG][2024-01-10 13:11:15,431][retry:0282] : Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
[DEBUG][2024-01-10 13:11:15,453][requests:0185] : Making request: POST https://oauth2.googleapis.com/token
[DEBUG][2024-01-10 13:11:15,456][connectionpool:1052] : Starting new HTTPS connection (1): oauth2.googleapis.com:443
[DEBUG][2024-01-10 13:11:1

In [4]:
import uuid
import numpy as np
df = pd.read_csv("./spreadspoke_scores.csv", header=0)

df = df[df["schedule_season"] > 1978]


df['uuid'] = [uuid.uuid4() for _ in range(len(df.index))]
df = df.astype({"uuid": "string"})


games_df = df.drop(columns=["weather_detail", "stadium_neutral", "over_under_line", "spread_favorite", "team_favorite_id"])

games_df.dropna(subset=["score_home", "score_away"], inplace=True)


games_df = games_df.rename(columns={"weather_humidity": "humidity_%", "weather_wind_mph": "wind_mph", "weather_temperature": "temperature_F", "schedule_date": "date", "schedule_season": "season", "schedule_week": "week", "schedule_playoff": "playoff"})

games_df["date"] = pd.to_datetime(games_df["date"])

games_df = games_df.astype({"score_home": int, "score_away": int, "temperature_F": int, "wind_mph": int, "humidity_%": int}, errors="ignore")

display(games_df)
games_df.info()



,date,season,week,playoff,team_home,score_home,score_away,team_away,stadium,temperature_F,wind_mph,humidity_%,uuid
2501,1979-09-01,1979,1,False,Tampa Bay Buccaneers,31,16,Detroit Lions,Houlihan's Stadium,79.0,9.0,87.0,1ea56e4b-c347-4dfe-873c-b54f96741602
2502,1979-09-02,1979,1,False,Buffalo Bills,7,9,Miami Dolphins,Ralph Wilson Stadium,74.0,15.0,74.0,84173a19-490d-4e1e-8105-5a7ca82fc0e6
2503,1979-09-02,1979,1,False,Chicago Bears,6,3,Green Bay Packers,Soldier Field,78.0,11.0,68.0,7bd09040-8e28-4cf9-98b3-c07e8127bba9
2504,1979-09-02,1979,1,False,Denver Broncos,10,0,Cincinnati Bengals,Mile High Stadium,69.0,6.0,38.0,3c997885-5a3d-4146-82b7-be620810cfb9
2505,1979-09-02,1979,1,False,Kansas City Chiefs,14,0,Baltimore Colts,Arrowhead Stadium,76.0,8.0,71.0,a2139538-e59d-4b1b-8a0f-2a903e54bb3e
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13751,2023-12-24,2023,16,False,Tampa Bay Buccaneers,30,12,Jacksonville Jaguars,Raymond James Stadium,NaN,NaN,NaN,817f8b24-754f-4aea-8a94-8404d55d3a3d
13752,2023-12-24,2023,16,False,Tennessee Titans,17,20,Seattle Seahawks,Nissan Stadium,NaN,NaN,NaN,172fc40d-f019-468a-a528-aa59bf308c28
13753,2023-12-25,2023,16,False,Kansas City Chiefs,14,20,Las Vegas Raiders,GEHA Field at Arrowhead Stadium,NaN,NaN,NaN,0f2b2264-7c2f-4c7e-9dbd-0b18dc21233f
13754,2023-12-25,2023,16,False,Philadelphia Eagles,33,25,New York Giants,Lincoln Financial Field,NaN,NaN,NaN,2b074e09-0691-4985-9d43-747988ee25a2


<class 'pandas.core.frame.DataFrame'>
Index: 11255 entries, 2501 to 13755
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           11255 non-null  datetime64[ns]
 1   season         11255 non-null  int64         
 2   week           11255 non-null  object        
 3   playoff        11255 non-null  bool          
 4   team_home      11255 non-null  object        
 5   score_home     11255 non-null  int64         
 6   score_away     11255 non-null  int64         
 7   team_away      11255 non-null  object        
 8   stadium        11255 non-null  object        
 9   temperature_F  10130 non-null  float64       
 10  wind_mph       10114 non-null  float64       
 11  humidity_%     6358 non-null   float64       
 12  uuid           11255 non-null  string        
dtypes: bool(1), datetime64[ns](1), float64(3), int64(3), object(4), string(1)
memory usage: 1.1+ MB


In [8]:
games_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['games']['table_name']}"
games_schema = schema=TABLE_METADATA['games']['schema']

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """

    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"


    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result() 

    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

load_table(games_df, client, games_table_name, games_schema)

[INFO ][2024-01-10 13:20:45,954][898524737:0032] : loading table: 'team-week-10.nfl_sports_betting.dim_games'
[DEBUG][2024-01-10 13:20:46,981][connectionpool:0546] : https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/team-week-10/jobs?uploadType=multipart HTTP/1.1" 200 2800
[DEBUG][2024-01-10 13:20:47,088][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/team-week-10/jobs/4a154ceb-916f-4437-a930-b6d04527e582?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2024-01-10 13:20:47,089][retry:0233] : Retrying due to , sleeping 0.7s ...
[DEBUG][2024-01-10 13:20:47,906][connectionpool:0546] : https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/team-week-10/jobs/4a154ceb-916f-4437-a930-b6d04527e582?location=US&prettyPrint=false HTTP/1.1" 200 None
[DEBUG][2024-01-10 13:20:47,907][retry:0233] : Retrying due to , sleeping 0.2s ...
[DEBUG][2024-01-10 13:20:48,210][connectionpool:0546] : https://bigquery.googleapis.com:443 "

In [9]:
from google.cloud import bigquery

client = bigquery.Client()
print(f"BigQuery Project: {client.project}")

# list datasets
print("Listing datasets:")

for dataset in client.list_datasets():
    dataset_id = dataset.dataset_id
    print(f"dataset id: `{dataset_id}`, full_name: `{dataset.full_dataset_id}`, labels (tags): {dataset.labels}")

[DEBUG][2024-01-10 13:22:26,623][_default:0255] : Checking /home/jarvismwynn/.creds/jarvis.json for explicit credentials as part of auth process...
[DEBUG][2024-01-10 13:22:26,626][_default:0255] : Checking /home/jarvismwynn/.creds/jarvis.json for explicit credentials as part of auth process...
BigQuery Project: team-week-10
Listing datasets:
[DEBUG][2024-01-10 13:22:26,628][retry:0282] : Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
[DEBUG][2024-01-10 13:22:26,651][requests:0185] : Making request: POST https://oauth2.googleapis.com/token
[DEBUG][2024-01-10 13:22:26,652][connectionpool:1052] : Starting new HTTPS connection (1): oauth2.googleapis.com:443
[DEBUG][2024-01-10 13:22:26,744][connectionpool:0546] : https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
[DEBUG][2024-01-10 13:22:26,746][connectionpool:1052] : Starting new HTTPS connection (1): bigquery.googleapis.com:443
[DEBUG][2024-01-10 13:22:26,985][connectionpo